<a href="https://colab.research.google.com/github/rikunosuke/django-csv/blob/main/CS%E3%82%B5%E3%83%9D%E3%83%BC%E3%83%88%E3%83%84%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに
本ツールは主に見積もりや前処理時に必要な  
簡易な動画・画像編集を行えるツールです。  

アノテーション情報を利用した画像処理を行いたいケースは  
別の[Fast Label SDKツール](https://colab.research.google.com/drive/1jiB5E53PEgbUTcpjLp7c9Lc13Haw2hB1?usp=sharing)をご利用ください。

### 利用上の注意
#### 1. 大量のファイル入出力を行う際のフォルダ指定
Google Drive上ではなくColab上のフォルダで作業を実施してください。  
Google DriveとはAPIでやり取りしているため、処理がかなり遅くなります。  
Colab上のフォルダからダウンロードしたい場合は  
CSサポートツールのzip機能でzipファイルを作成しダウンロードしてください。  
※Colab上のフォルダは下記利用時間の制限で初期化されるため、大量件数を扱いたいときにはdemoアカウントを利用ください。
<br><br>
#### 2. 利用時間の制限
通常のGoogleアカウントでは、以下の利用制限があります。
- セッションが切れた後、90分で初期化
- 初回接続後(画面を開いて接続後)が12時間で初期化

長時間利用したい、または、重たい処理を実行したい場合は<br>
demo1@fastlabel.ai のアカウントで実行してください(パスワードは内田まで)。<br>
上記アカウントでは、最大24時間まで、バックグラウンド実行も可能です。

# ①環境セットアップ
最初の起動時には一度実行してください。

In [ ]:
#@title <font size="4"><i>Colabの初期化・ライブラリのインストール</font> { vertical-output: true }
from IPython.display import clear_output
from IPython.utils import io
import os
import glob
import subprocess
import tqdm.notebook
import urllib
import time

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

try:
  print('インストール実行中...\n')
  with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      # fastlabel SDK
      !pip install fastlabel
      pbar.update(20)
      
      # ffmpeg
      HOME = os.path.expanduser("~")
      if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
          hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
          urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

      from ttmg import (
          loadingAn,
          textAn,
      )

      ## git clone
      loadingAn(name="lds")
      textAn("Cloning Repositories...", ty='twg')
      !git clone https://github.com/XniceCraft/ffmpeg-colab.git
      
      ## install
      pbar.update(30)
      !chmod 755 ./ffmpeg-colab/install
      textAn("Installing FFmpeg...", ty='twg')
      !./ffmpeg-colab/install
      clear_output()
      print('Installation finished!')
      !rm -fr /content/ffmpeg-colab
      
      pbar.update(30)

      # ffmpeg-python
      !pip install ffmpeg-python

      # pdf2image
      !apt-get install poppler-utils
      !pip install pdf2image 
      pbar.update(20)
except subprocess.CalledProcessError:
  print(captured)
  raise
finally:
  time.sleep(1)
  print("\nインストール完了！", flush=True)
  print("※セッションがクラッシュしたメッセージは無視してそのまま使用してください", flush=True)
  time.sleep(1)
  os._exit(00) # インストールしたライブラリをランタイムに反映するため

In [ ]:
#@title <font size="4"><i>Google Driveに接続</font>
import ffmpeg
import os
import glob
import math
import time
from tqdm.notebook import tqdm
from google.colab import drive
drive.mount('/content/drive')

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

def get_file_paths(dir, recursive = True, filter_ext_list = []):
    file_paths = []

    target_file_candidate_paths = glob.glob(os.path.join(dir, "**"), recursive=recursive)
    
    for target_file_candidate_path in target_file_candidate_paths:
        if not os.path.isfile(target_file_candidate_path):
            continue
        
        _, ext = os.path.splitext(os.path.basename(target_file_candidate_path))
        if len(filter_ext_list) > 0 and ext.lower() not in filter_ext_list:
            continue
        
        file_paths.append(target_file_candidate_path)

    file_paths.sort()

    return file_paths

def get_output_path(input_dir, output_dir, file_path, file_name = "", keep_directory = False):
    if not file_name:
        file_name = os.path.basename(file_path)

    if keep_directory:
        target_dir = os.path.dirname(file_path).replace(input_dir, '', 1)
        if target_dir.startswith('/'):
            target_dir = target_dir.replace('/', '', 1)
        target_output_dir = os.path.join(output_dir, target_dir)
        os.makedirs(target_output_dir, exist_ok=True)
        return os.path.join(target_output_dir, file_name)
    else:
        return os.path.join(output_dir, file_name)

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None
    
    @staticmethod
    def start():
        timer = Timer()
        """Start a new timer"""
        timer._start_time = time.perf_counter()
        return timer

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        print(f"\nElapsed time: {elapsed_time:0.4f} seconds")
        print('Finished!!')


# ②ツール実行

## 【動画編集】

In [ ]:
#@title <font size="4">឵឵<i>動画の圧縮</font>{ vertical-output: true }
#@markdown #### 実行の前に
#@markdown このツールの実行には時間がかかります。
#@markdown 
#@markdown 参考：
#@markdown 
#@markdown 15分 (約300MB) の動画で20分ほど
#@markdown 
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_dir：入力ファイルのGoogle Driveフォルダパス
#@markdown output_dir：出力ファイルのGoogle Driveフォルダパス (事前の作成は不要)
#@markdown keep_directory:入力フォルダの階層を維持して出力をするか、指定フォルダ直下に出力するか
#@markdown ```

input_dir = "drive/MyDrive/input" #@param {type:"string"}
output_dir = "drive/MyDrive/output"#@param {type:"string"}
keep_directory = "\u6307\u5B9A\u30D5\u30A9\u30EB\u30C0\u76F4\u4E0B\u306B\u51FA\u529B" #@param ["指定フォルダ直下に出力","入力フォルダの階層を維持して出力"]

timer = Timer.start()

os.makedirs(output_dir, exist_ok=True)
file_paths = get_file_paths(input_dir)

CRF = 33 # 一旦固定

with tqdm(total=len(file_paths), bar_format=TQDM_BAR_FORMAT) as pbar:
    for file_path in file_paths:
        print('Processing ', file_path)
        output_path = get_output_path(input_dir, output_dir, file_path, keep_directory = (keep_directory == '入力フォルダの階層を維持して出力'))
        out, _ = (
            ffmpeg.input(file_path).output(output_path, crf=CRF).run(overwrite_output=True)
        )
        pbar.update(1)

timer.stop()

In [ ]:
#@title <font size="4">឵឵<i>動画の分割</font>{ vertical-output: true }
#@markdown ##### 実行の前に
#@markdown このツールの実行には時間がかかります。
#@markdown 
#@markdown 参考：
#@markdown 
#@markdown 15分 (約300MB) の動画で、1分割についき15分ほど
#@markdown 
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_dir：入力ファイルのGoogle Driveフォルダパス
#@markdown output_dir：出力ファイルのGoogle Driveフォルダパス (事前の作成は不要)
#@markdown split_num：動画の分割数
#@markdown keep_directory:入力フォルダの階層を維持して出力をするか、指定フォルダ直下に出力するか
#@markdown ```

input_dir = "drive/MyDrive/input" #@param {type:"string"}
output_dir = "drive/MyDrive/output"#@param {type:"string"}
split_num = 3#@param {type:"number"}
# split_duration = None#@param {type:"number"}
keep_directory = "\u6307\u5B9A\u30D5\u30A9\u30EB\u30C0\u76F4\u4E0B\u306B\u51FA\u529B" #@param ["指定フォルダ直下に出力","入力フォルダの階層を維持して出力"]

timer = Timer.start()

os.makedirs(output_dir, exist_ok=True)
file_paths = get_file_paths(input_dir)

with tqdm(total=len(file_paths)*split_num, bar_format=TQDM_BAR_FORMAT) as pbar:
    for file_path in file_paths:
        print('Processing ', file_path)
        file_info = ffmpeg.probe(file_path)
        video_stream_info = next(
                filter(
                    lambda stream: stream["codec_type"] == "video",
                    file_info["streams"],
                ),
                None,
            )
        audio_stream_info = next(
            filter(
                lambda stream: stream["codec_type"] == "audio",
                file_info["streams"],
            ),
                None,
            )

        # ファイル情報
        frame_count = int(video_stream_info["nb_frames"])

        # １つ当たりのフレーム数を切り上げて計算
        split_frame_count=math.ceil(frame_count/split_num)
        fps = int(video_stream_info['r_frame_rate'].split('/')[0]) / int(video_stream_info['r_frame_rate'].split('/')[1])
 
        for i in range(split_num):
            start_frame = i * split_frame_count
            end_frame = (i + 1) * split_frame_count
            file_name = os.path.splitext(os.path.basename(file_path))[0] + f"_{i+1}" + os.path.splitext(file_path)[1]
            output_path = get_output_path(input_dir, output_dir, file_path, file_name, keep_directory = (keep_directory == '入力フォルダの階層を維持して出力'))
            
            input = ffmpeg.input(file_path)

            video_trim = input.trim(
                start_frame=start_frame, end_frame=end_frame
            ).setpts("PTS-STARTPTS")

            if audio_stream_info:
                audio = input.audio
                audio_trim = audio.filter(
                    "atrim", start=start_frame / fps, end=end_frame / fps
                ).filter("asetpts", "PTS-STARTPTS")
                
                joined = ffmpeg.concat(video_trim, audio_trim, v=1, a=1).node
                output = ffmpeg.output(joined[0], joined[1], output_path)
                output.run(overwrite_output=True)
            else:
                # 音声がない場合はvideoのみoutput
                video_trim.output(output_path).run(overwrite_output=True)
                
            pbar.update(1)

timer.stop()

In [ ]:
#@title <font size="4">឵឵<i>動画から画像の切り出し</font>{ vertical-output: true }
#@markdown #### 実行の前に
#@markdown このツールの実行には時間がかかります。
#@markdown 
#@markdown 参考：
#@markdown 
#@markdown 15分 (約300MB) の動画で30分ほど
#@markdown 
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_dir：入力ファイルのGoogle Driveフォルダパス
#@markdown output_dir：出力ファイルのGoogle Driveフォルダパス (事前の作成は不要)
#@markdown frame_step：何フレームごとに画像を切り出すか(全てのフレームで切り出す場合は1を指定)
#@markdown file_ext：変換後の画像ファイル拡張子 (例：png)
#@markdown keep_directory:入力フォルダの階層を維持して出力をするか、指定フォルダ直下に出力するか
#@markdown ```

input_dir = "drive/MyDrive/input" #@param {type:"string"}
output_dir = "drive/MyDrive/output"#@param {type:"string"}
frame_step = 10#@param {type:"number"}
file_ext = "png"#@param {type:"string"}
keep_directory = "\u6307\u5B9A\u30D5\u30A9\u30EB\u30C0\u76F4\u4E0B\u306B\u51FA\u529B" #@param ["指定フォルダ直下に出力","入力フォルダの階層を維持して出力"]

timer = Timer.start()

os.makedirs(output_dir, exist_ok=True)
file_paths = get_file_paths(input_dir)

with tqdm(total=len(file_paths), bar_format=TQDM_BAR_FORMAT) as pbar:
    for file_path in file_paths:
        print('Processing ', file_path)
        # ffmpeg.probeを実施して、動画のメタデータを取得
        video_info = ffmpeg.probe(file_path)
        r_frame_rate = video_info['streams'][0]['r_frame_rate']
        splited = os.path.splitext(os.path.basename(file_path))
        file_name = splited[0] + '_%d.' + file_ext
        output_path = get_output_path(input_dir, output_dir, file_path, file_name, keep_directory = (keep_directory == '入力フォルダの階層を維持して出力'))
        print(output_path)
        !ffmpeg -loglevel error -y -i "$file_path" -vf framestep="$frame_step" "$output_path"
        pbar.update(1)

timer.stop()

In [ ]:
#@title <font size="4">឵឵<i>動画のFPS変換</font>{ vertical-output: true }
#@markdown #### 実行の前に
#@markdown このツールの実行には時間がかかります。
#@markdown 
#@markdown 参考：
#@markdown 
#@markdown 15分 (約300MB) の動画で30分ほど
#@markdown 
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_dir：入力ファイルのGoogle Driveフォルダパス
#@markdown output_dir：出力ファイルのGoogle Driveフォルダパス (事前の作成は不要)
#@markdown fps：変換後のFPS (例：　１０FPSの動画にしたい場合は「10」を指定)
#@markdown keep_directory:入力フォルダの階層を維持して出力をするか、指定フォルダ直下に出力するか
#@markdown ```

input_dir = "drive/MyDrive/input" #@param {type:"string"}
output_dir = "drive/MyDrive/output"#@param {type:"string"}
fps = 2.5#@param {type:"number"}
keep_directory = "\u6307\u5B9A\u30D5\u30A9\u30EB\u30C0\u76F4\u4E0B\u306B\u51FA\u529B" #@param ["指定フォルダ直下に出力","入力フォルダの階層を維持して出力"]

timer = Timer.start()

os.makedirs(output_dir, exist_ok=True)
file_paths = get_file_paths(input_dir)

with tqdm(total=len(file_paths), bar_format=TQDM_BAR_FORMAT) as pbar:
    for file_path in file_paths:
        print('Processing ', file_path)
        output_path = get_output_path(input_dir, output_dir, file_path, keep_directory = (keep_directory == '入力フォルダの階層を維持して出力'))
        out, _ = (
            ffmpeg.input(file_path).filter('fps', fps=fps, round='up').output(output_path).run(overwrite_output=True)
        )
        pbar.update(1)

timer.stop()

In [ ]:
#@title <font size="4">឵឵<i>動画情報の確認</font>{ vertical-output: true }
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_dir：入力ファイルのGoogle Driveフォルダパス
#@markdown ```

input_dir = "/content/drive/Shareddrives/FastLabel_All/000_CLIENT/\uFF84)_\u30C8\u30E8\u30BF\u81EA\u52D5\u8ECA\u682A\u5F0F\u4F1A\u793E/02_\u30D5\u309A\u30ED\u30B7\u3099\u30A7\u30AF\u30C8\u8CC7\u6599/Infotech/202212_\u4EA4\u901A\u52D5\u753B/\u5909\u63DB\u5F8C\u30C6\u3099\u30FC\u30BF/video1/\u5206\u5272\u524D_2.5fps" #@param {type:"string"}

timer = Timer.start()

file_paths = get_file_paths(input_dir)

for file_path in file_paths:
  if ".mp4" not in file_path:
    continue
  file_info = ffmpeg.probe(file_path)
  video_stream_info = next(
          filter(
              lambda stream: stream["codec_type"] == "video",
              file_info["streams"],
          ),
          None,
      )
    
  frame_count = int(video_stream_info["nb_frames"])
  duration = float(video_stream_info["duration"])
  fps = int(video_stream_info['r_frame_rate'].split('/')[0]) / int(video_stream_info['r_frame_rate'].split('/')[1])
  print(os.path.basename(file_path),"\n   fps  ",fps,"\n   duration  ",duration,"\n   frame_count  ",frame_count)

timer.stop()

## 【画像編集】

In [ ]:
#@title <font size="4">឵឵<i>PDFから画像に変換</font>{ vertical-output: true }
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_dir：入力ファイルのGoogle Driveフォルダパス
#@markdown output_dir：出力ファイルのGoogle Driveフォルダパス (事前の作成は不要)
#@markdown file_ext：変換後の画像ファイル拡張子 (例：png)
#@markdown keep_directory:入力フォルダの階層を維持して出力をするか、指定フォルダ直下に出力するか
#@markdown ```

from pdf2image import convert_from_path

input_dir = "drive/MyDrive/input" #@param {type:"string"}
output_dir = "drive/MyDrive/output"#@param {type:"string"}
file_ext = "png"#@param {type:"string"}
keep_directory = "\u6307\u5B9A\u30D5\u30A9\u30EB\u30C0\u76F4\u4E0B\u306B\u51FA\u529B" #@param ["指定フォルダ直下に出力","入力フォルダの階層を維持して出力"]

timer = Timer.start()

os.makedirs(output_dir, exist_ok=True)
file_paths = get_file_paths(input_dir)

for file_path in file_paths:
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    output_path = get_output_path(input_dir, output_dir, file_path, file_name, keep_directory = (keep_directory == '入力フォルダの階層を維持して出力'))
    images = convert_from_path(file_path)
    for index, image in enumerate(images):
        image.save(output_path + '-{}'.format(index + 1) + '.' + file_ext, file_ext)

timer.stop()

In [ ]:
#@title <font size="4">឵឵<i>画像のリサイズ</font>{ vertical-output: true }
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_dir：入力ファイルのGoogle Driveフォルダパス
#@markdown output_dir：出力ファイルのGoogle Driveフォルダパス (事前の作成は不要)
#@markdown width：横のサイズ
#@markdown height：縦のサイズ
#@markdown ※アスペクト比を維持する場合は、縦横のどちらかを指定。
#@markdown keep_directory:入力フォルダの階層を維持して出力をするか、指定フォルダ直下に出力するか
#@markdown ```

input_dir = "drive/MyDrive/input" #@param {type:"string"}
output_dir = "drive/MyDrive/output"#@param {type:"string"}
width = 6000#@param {type:"number"}
height = None#@param {type:"number"}
keep_directory = "\u5165\u529B\u30D5\u30A9\u30EB\u30C0\u306E\u968E\u5C64\u3092\u7DAD\u6301\u3057\u3066\u51FA\u529B" #@param ["指定フォルダ直下に出力","入力フォルダの階層を維持して出力"]


timer = Timer.start()

if (not width and not height):
  raise ValueError("widthかheightのどちらかを指定してください")

os.makedirs(output_dir, exist_ok=True)
file_paths = get_file_paths(input_dir)

for file_path in file_paths:
    output_path = get_output_path(input_dir, output_dir, file_path, keep_directory = (keep_directory == '入力フォルダの階層を維持して出力'))
    print(output_path)
    width = width if width else -1
    height = height if height else -1
    out, _ = (
        ffmpeg.input(file_path).filter('scale', width, height).output(output_path).run(overwrite_output=True)
    )

timer.stop()

In [ ]:
#@title <font size="4">឵឵<i>画像の拡張子変換</font>{ vertical-output: true }
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_dir：入力ファイルのGoogle Driveフォルダパス
#@markdown output_dir：出力ファイルのGoogle Driveフォルダパス (事前の作成は不要)
#@markdown file_ext：変換後のファイル拡張子
#@markdown keep_directory:入力フォルダの階層を維持して出力をするか、指定フォルダ直下に出力するか
#@markdown ```

from PIL import Image

input_dir = "drive/MyDrive/input" #@param {type:"string"}
output_dir = "drive/MyDrive/output"#@param {type:"string"}
file_ext = "png"#@param {type:"string"}
keep_directory = "\u5165\u529B\u30D5\u30A9\u30EB\u30C0\u306E\u968E\u5C64\u3092\u7DAD\u6301\u3057\u3066\u51FA\u529B" #@param ["指定フォルダ直下に出力","入力フォルダの階層を維持して出力"]

timer = Timer.start()

os.makedirs(output_dir, exist_ok=True)
file_paths = get_file_paths(dir=input_dir)

with tqdm(total=len(file_paths), bar_format=TQDM_BAR_FORMAT) as pbar:
    for file_path in file_paths:
        img = Image.open(file_path)
        file_name = os.path.splitext(os.path.basename(file_path))[0] + "." + file_ext
        output_path = get_output_path(input_dir, output_dir, file_path, file_name, keep_directory = (keep_directory == '入力フォルダの階層を維持して出力'))
        print(output_path)
        img.save(output_path)
        pbar.update(1)

timer.stop()

In [ ]:
#@title 画像から動画を作成
#@markdown ## パラメータ
#@markdown ```
#@markdown  _INPUT_DIR：入力ファイルのGoogle Driveファイルパス
#@markdown  IMAGE_DIR：対象のディレクトリの、画像が入っているディレクトリの名前（空欄可能）
#@markdown  _OUTPUT_DIR：出力ファイルのGoogle Driveファイルパス(事前の作成は不要)
#@markdown  TARGET_EXTENSIONS：ターゲットにしたい拡張子をカンマ区切りで入力 （大文字・小文字は区別されない）
#@markdown  SEQUENCE_PATTERN：画像の連番を取得するための正規表現。連番の部分を **()** で囲む
#@markdown  FRAME_RATE：出力される動画のフレームレート
#@markdown ```
#@markdown 
#@markdown ### 入力画像について
#@markdown ```
#@markdown _INPUT_DIR 内のフォルダが対象となり、動画名はフォルダの名前になる。
#@markdown 例)
#@markdown _INPUT_DIR：drive/MyDrive/input
#@markdown IMAGE_DIR：data
#@markdown の時、
#@markdown **drive/MyDrive/input/テスト動画/data/\*.jpeg** が読み込まれ、**テスト動画.mp4** が作成される
#@markdown ```

import re
from pathlib import Path

import ffmpeg as fp
from concurrent.futures import ThreadPoolExecutor
import tempfile

_INPUT_DIR = "drive/MyDrive/video/input"  # @param {type:"string"}
INPUT_DIR = Path(_INPUT_DIR)

IMAGE_DIR = "data"  # @param {type:"string"}

_OUTPUT_DIR = "drive/MyDrive/video/output"  # @param {type:"string"}
OUTPUT_DIR = Path(_OUTPUT_DIR)

_INPUT_TARGET_EXTENSIONS = "jpeg jpg png"  # @param {type:"string"}
INPUT_TARGET_EXTENSIONS = (
    _INPUT_TARGET_EXTENSIONS
    .replace(" ", "　")
    .split()
)

SEQUENCE_PATTERN = r"\D*(\d+)\D*.\w{3,4}"  # @param {type:"string"}

FRAME_RATE = 3  # @param {type:"number"}

TARGET_EXTENSIONS = tuple(
    f".{extension.replace('.', '')}" for extension in INPUT_TARGET_EXTENSIONS
)

def get_sequence_number(filename: str) -> int:
    if m := re.search(SEQUENCE_PATTERN, filename):
        return int(m.groups()[0])

    raise ValueError(f"ファイル名 '{filename}' から連続する数値を検出できませんでした")


def check_if_targe_file(file_path: Path) -> bool:
    return file_path.is_file() and file_path.suffix.lower() in TARGET_EXTENSIONS


def get_all_target_file_paths(dir_path: Path) -> list[Path]:
    file_paths = [
        p for p in dir_path.glob("*") if check_if_targe_file(p)
    ]
    # print(f"target count: {len(file_paths)}")
    return sorted(
        file_paths, key=lambda x: get_sequence_number(x.name)
    )


def create_movie(dir_name: str) -> None:
    OUTPUT_DIR.mkdir(exist_ok=True)

    try:
        with tempfile.TemporaryDirectory() as str_temp_dir:
            file_list_path = f"{str_temp_dir}/{dir_name}.txt"
            file_paths = get_all_target_file_paths(INPUT_DIR / dir_name / IMAGE_DIR)

            with open(file_list_path, mode="w", encoding="utf-8") as f:
                f.write("\n".join([f"file '{line.absolute()}'" for line in file_paths]))

            print(f"{OUTPUT_DIR} に {dir_name}.mp4 を書き出し中 ...")
            fp.input(file_list_path, f="concat", safe=0, r=FRAME_RATE).output(
                str(OUTPUT_DIR / f"{dir_name}.mp4")
            ).run(overwrite_output=True, quiet=False, capture_stdout=True, capture_stderr=True)
            print("完了")
    except Exception as e:
        print('stdout:', e.stdout.decode('utf8'))
        print('stderr:', e.stderr.decode('utf8'))


dirs = [p.stem for p in Path(INPUT_DIR).iterdir()]

timer = Timer.start()
with ThreadPoolExecutor() as executor:
    executor.map(create_movie, dirs)

timer.stop()


## 【ファイル・フォルダ操作】


In [ ]:
#@title <font size="4">឵឵<i>フォルダのzip圧縮</font>{ vertical-output: true }
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_dir：入力ファイルのGoogle Driveフォルダパス
#@markdown ```

input_dir = "drive/MyDrive/input" #@param {type:"string"}

timer = Timer.start()

last_dir_name = os.path.basename(os.path.normpath(input_dir))
output_zip_file_name = last_dir_name + '.zip'
parent_dir = os.path.abspath(os.path.join(input_dir, os.pardir))

!cd "$parent_dir" && zip "$output_zip_file_name" -r "$last_dir_name"

timer.stop()

In [ ]:
#@title <font size="4">឵឵<i>フォルダのzip解凍</font>{ vertical-output: true }
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_file_path：zipファイルのGoogle Driveファイルパス
#@markdown output_dir：(オプション)出力フォルダパス(指定しない場合はinput_fileが存在するフォルダ直下に解凍されます)
#@markdown ```

input_file_path = "/work/CS\u30B5\u30DB\u309A\u30FC\u30C8\u30C4\u30FC\u30EB.ipynb.zip" #@param {type:"string"}
output_dir = "/work/"#@param {type:"string"}

timer = Timer.start()

if not output_dir:
    output_dir = os.path.abspath(os.path.join(input_file_path, os.pardir)) #親ディレクトリを指定

!unzip -Ocp932 -o "$input_file_path" -d "$output_dir"

timer.stop()

In [ ]:
#@title <font size="4">឵឵<i>フォルダ階層の変更</font>{ vertical-output: true }
#@markdown スプレッドシートのフォルダマッピング設定を元に、GoogleDrive上のフォルダをコピーするツールです。　　<br>
#@markdown ※注：親・子の両方にマッピング設定があると意図した動作になりません。<br>
#@markdown ```
#@markdown (例) 以下の構造では、「folder1」と「folder1-1」に設定があるのはNG、「folder1」と「folder2」、または「folder1-1」と「folder2」はOKです。
#@markdown root
#@markdown 　　|--folder1
#@markdown 　　　　|--folder1-1
#@markdown 　　|--folder2
#@markdown ```
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_dir：Google Driveのフォルダ
#@markdown ss_url：スプレッドシートのURL　
#@markdown sheet_name：マッピング設定情報のシート名　
#@markdown ```
#@markdown <small>[参考：入力するスプレッドシートのサンプル](https://docs.google.com/spreadsheets/d/10QCOc3OxUmRt-sTcoBiyY8x0zmISDthd0Gfd2kfjHYE/edit?usp=sharing)</small>


import gspread
from google.auth import default
from google.colab import auth
import shutil

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

FROM_COL = 'from'
TO_COL = 'to'

input_dir = "/content/drive/MyDrive/input" #@param {type:"string"}
ss_url = "https://docs.google.com/spreadsheets/d/10QCOc3OxUmRt-sTcoBiyY8x0zmISDthd0Gfd2kfjHYE/edit#gid=0"#@param {type:"string"}
sheet_name = "mapping"#@param {type:"string"}


def execute(dirs, mappings):
    if len(mappings) == 0:
        raise ValueError('シートにデータがありません')

    columns = list(mappings[0].keys())
    from_col = next((key for key in columns if key.lower() == FROM_COL), None)
    to_col = next((key for key in columns if key.lower() == TO_COL), None)
    
    if not (from_col and to_col):
        raise ValueError('スプレッドシートに' + FROM_COL + 'と' + TO_COL + '列がありません')
    
    for dir in dirs:
        index = next((index for index, mapping in enumerate(mappings) if mapping['from'] == dir),  None)
        # print(index)
        if index is not None: 
            mapping = mappings[index]
            to = mapping['to']
            print("WARN: Copy folder from ", dir, " to ", to)
            if os.path.exists(to):
                print("To dir ", to, " exists. Copy skipped.")
                continue
            shutil.copytree(dir, to)



def get_dir_paths(root_dir):
    result = []
    for root, dirs, files in os.walk(top=root_dir):
        for dir in dirs:
            dirPath = os.path.join(root, dir)
            result.append(dirPath)
    return result


timer = Timer.start()

# initialize params
workbook = gc.open_by_url(ss_url)
sheet = workbook.worksheet(sheet_name)

dirs = get_dir_paths(input_dir)
print('Target dirs:', dirs)

# main
mappings = sheet.get_all_records()
execute(dirs, mappings)

timer.stop()

In [ ]:
#@title <font size="4">឵឵<i>Fast Labelツールにアップロード可能なサイズでzipファイル作成</font>{ vertical-output: true }

import zipfile

def bytesto(bytes, to, bsize=1024): 
    a = {'k' : 1, 'm': 2, 'g' : 3, 't' : 4, 'p' : 5, 'e' : 6 }
    r = float(bytes)
    return bytes / (bsize ** a[to])

input_dir = "drive/MyDrive/input" #@param {type:"string"}
output_dir = "drive/MyDrive/output" #@param {type:"string"}

# initialize
timer = Timer.start()
os.makedirs(output_dir, exist_ok=True)

input_dir_name = os.path.basename(os.path.normpath(input_dir))
original_output_zip_path = os.path.join(output_dir, input_dir_name + '.zip')

file_paths = get_file_paths(dir=input_dir)

# main
zip_index = 0
total_file_size = 0
f = None

with tqdm(total=len(file_paths), bar_format=TQDM_BAR_FORMAT) as pbar:
    for i, file_path in enumerate(file_paths):
        if not f:
            output_zip_path = original_output_zip_path if zip_index == 0 else os.path.splitext(original_output_zip_path)[0] + "_{}".format(zip_index + 1) + '.zip'
            f = zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED)
            print('Creating zip file:', output_zip_path)
            
        file_size = os.path.getsize(file_path)
        file_name = file_path.replace(input_dir, "")
        f.write(file_path, arcname=file_name)
        total_file_size += file_size
        
        if bytesto(total_file_size, 'g') >= 3:
            print('Total file size exceeds limit at file index :', i, '. Change zip file.')
            f.close()
            f = None
            zip_index += 1
            total_file_size = 0

        pbar.update(1)

if f:
    f.close()

timer.stop()

In [ ]:
#@title <font size="4">឵឵<i>Fast Labelツールにアップロード可能な件数にアノテーションJSONファイル分割(サイズは考慮)</font>{ vertical-output: true }
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_file_path：入力ファイルのGoogle Driveファイルパス
#@markdown output_dir：出力ファイルのGoogle Driveフォルダパス (事前の作成は不要)
#@markdown annotations_count：１つのJSONファイルあたりのアノテーション数
#@markdown ```

import json

input_file_path = "/content/drive/MyDrive/input/annotations.json" #@param {type:"string"}
output_dir = "/content/drive/MyDrive/output" #@param {type:"string"}
annotations_count = 550#@param {type:"number"}

timer = Timer.start()

os.makedirs(output_dir,exist_ok=True)

tasks = json.load(open(input_file_path))
print(len(tasks))
annotation_count=0
file_count=0

new_tasks=[]
for task in tasks:
  task_name=task["name"]
  annotations = task["annotations"]
  annotation_count+=len(annotations)
  new_tasks.append(task)

  if annotation_count>annotations_count:
    output_path=os.path.join(output_dir,f"annotations_{file_count}.json")
    with open(output_path, mode='wt', encoding='utf-8') as file:
      json.dump(new_tasks, file, ensure_ascii=False, indent=2)
    print(file_count,annotation_count)
    new_tasks=[]
    file_count+=1
    annotation_count=0

output_path=os.path.join(output_dir,f"annotations_{file_count}.json")
with open(output_path, mode='wt', encoding='utf-8') as file:
  json.dump(new_tasks, file, ensure_ascii=False, indent=2)

print(file_count,annotation_count)

timer.stop()

In [ ]:
#@title <font size="4">឵឵<i>ファイル名に連番付与</font>{ vertical-output: true }
#@markdown ###### パラメータ
#@markdown ```
#@markdown input_dir：入力ファイルのGoogle Driveファイルパス
#@markdown output_dir：出力ファイルのGoogle Driveフォルダパス (事前の作成は不要)
#@markdown digits：連番の桁数
#@markdown search_regex：検索文字列(指定された場合：()で囲まれた指定文字列を置換して連番を振る、指定されなかった場合：単純にファイル名の末尾に連番を振る)
#@markdown prefix：連番のプレフィックス
#@markdown suffix：連番のサフィックス
#@markdown keep_directory:入力フォルダの階層を維持して出力をするか、指定フォルダ直下に出力するか
#@markdown ```

import re
import shutil

input_dir = "drive/MyDrive/input" #@param {type:"string"}
output_dir = "drive/MyDrive/output" #@param {type:"string"}
digits = 3#@param {type:"number"}
#search_regex = "([0-9]{2})\\.wav$"#@param {type:"string"}
search_regex = ""#@param {type:"string"}

prefix = "s"#@param {type:"string"}
suffix = "suf"#@param {type:"string"}
keep_directory = "\u6307\u5B9A\u30D5\u30A9\u30EB\u30C0\u76F4\u4E0B\u306B\u51FA\u529B" #@param ["指定フォルダ直下に出力","入力フォルダの階層を維持して出力"]

def replace_number(file_name, digits, m):
    target_str = m.group(1)
    return m.group(0).replace(target_str, prefix + f"{int(target_str):0{digits}}" + suffix)

timer = Timer.start()

file_paths = get_file_paths(input_dir)

for index, file_path in enumerate(file_paths):
    if search_regex:
        file_name = os.path.basename(file_path)
        m = re.search(search_regex, file_name)
        if not m:
            print('No target word found:', file_name)
            continue

        replaced = re.sub(search_regex, lambda m: replace_number(file_name, digits, m), file_name)
        output_file_path = get_output_path(input_dir, output_dir, file_path, replaced, keep_directory = (keep_directory == '入力フォルダの階層を維持して出力'))
    else:
        file_name, ext = os.path.splitext(os.path.basename(file_path))
        file_no = index + 1
        file_name = file_name + prefix + f"{file_no:0{digits}}" + suffix + ext
        output_file_path = get_output_path(input_dir, output_dir, file_path, file_name, keep_directory = (keep_directory == '入力フォルダの階層を維持して出力'))
        
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    shutil.copy(file_path, output_file_path)
    
timer.stop()